In [1]:
import matplotlib.pyplot as plt
import uproot
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.model_selection import train_test_split

from datetime import datetime
import sys
sys.path.insert(1, '/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/util_funcs/')
from pandas_colFuncs import whichBisSig_NAHS, B_ID, ancestor_below_B

In [2]:
now = datetime.now()
print("time at start =", now)

time at start = 2021-11-01 14:34:34.115311


In [3]:
save_data = False
save_preprocessedDataframe = True
tmp_data = False
take_subset = False
subset_size = 100000

delete_top_numFSP_evts=True

In [4]:
nfs_path = "/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/"

data_subdir = "NAHSA_Gmodes_1/"
root_subdir = "NAHS_allEvts_twoSubs/"  

root_path = nfs_path + "NAHS/onlineDataProd/" + root_subdir 
#root_path = "/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/"

In [5]:
merged = ""
sub = "sub0/"

In [6]:
fileHc = uproot.open(root_path + sub + merged + "allHc.root:variables")

In [7]:
df_Hc = fileHc.arrays(library="pd")

In [ ]:
names = ["gammas","electrons","pions","kaons","muons"]
dfs = []
for name in names:
    filename=root_path + sub  + merged + "{}.root:variables".format(name)
    print(filename)
    tmpFileFSPs = uproot.open(filename)
    df_tmp = tmpFileFSPs.arrays(library="pd")
    dfs.append(df_tmp)

/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/NAHS_allEvts_twoSubs/sub0/gammas.root:variables


In [ ]:
df_FSPs = pd.concat(dfs)

In [ ]:
# delete FSPs for which no Y4S file entry was found
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Hc["__event__"])]

In [ ]:
df_FSPs.keys()

In [ ]:
print(df_FSPs.shape[0])
print(df_Hc.shape[0])
print(df_Hc["__event__"].nunique())

## check the subset of (B-sig side) D*-> D0 (-> kpi, kpipi,...) pi  

In [53]:
df_Hc.keys()

Index(['__experiment__', '__run__', '__event__', '__candidate__',
       '__ncandidates__', '__weight__', 'extraInfo__boSignalProbability__bc',
       'isSignalAcceptMissingGamma', 'PDG', 'genUp4S_uniqParID_0',
       ...
       'z', 'x_uncertainty', 'y_uncertainty', 'z_uncertainty',
       'uniqueParticleIdentifier', 'genParticleID', 'Bsig_uniqParID',
       'HcMother_uniqParID', 'Hc_genParticleID', 'Hc_MotherPDG'],
      dtype='object', length=103)

In [109]:
# select only these D0 modes and D*+ -> D0 pi
#D0:kpi 
#D0:kpipi pi 
#D0:kpipi0     
#D0:kpipi pi pi0 

In [17]:
df_Hc['Bsig_uniqParID'] = df_Hc.apply(whichBisSig_NAHS, axis=1)

In [18]:
df_Hc['Bsig_uniqParID'].value_counts()

83886081.00    1707200
83886082.00    1539979
Name: Bsig_uniqParID, dtype: int64

In [22]:
pd.options.display.float_format = '{:20.2f}'.format
df_Hc = df_Hc[((df_Hc["Bsig_uniqParID"] == 83886081.0)  &
            (df_Hc["genUp4S_PDG_0_0_0"].abs() == 421.0)  & # 421=D0
            (df_Hc["genUp4S_PDG_0_0_1"].abs() == 211.0)  & # 211=pion
            (df_Hc["genUp4S_PDG_0_0_2"].isna() == True)  & # no particle 
            (df_Hc["genUp4S_PDG_0_0_0_0"].abs() == 321.0)  & # 321=kaon 
            (df_Hc["genUp4S_PDG_0_0_0_1"].abs() == 211.0)  & # 211=pion
            ((df_Hc["genUp4S_PDG_0_0_0_2"].abs() == 211.0)  | # 211=pion
            (df_Hc["genUp4S_PDG_0_0_0_2"].isna() == True) | # OR no particle
            (df_Hc["genUp4S_PDG_0_0_0_2"].abs() == 111.0)) & # OR 111=pi0
            ((df_Hc["genUp4S_PDG_0_0_0_3"].abs() == 211.0)  | # 211=pion
            (df_Hc["genUp4S_PDG_0_0_0_3"].isna() == True)) & # OR no particle
            ((df_Hc["genUp4S_PDG_0_0_0_4"].abs() == 111.0)  | # 111=pi0
            (df_Hc["genUp4S_PDG_0_0_0_4"].isna() == True)) & # OR no particle
            (df_Hc["genUp4S_PDG_0_0_0_5"].isna() == True)) # no particle
              |
        ((df_Hc["Bsig_uniqParID"] == 83886082.0)  &
            (df_Hc["genUp4S_PDG_1_0_0"].abs() == 421.0)  & # 421=D0
            (df_Hc["genUp4S_PDG_1_0_1"].abs() == 211.0)  & # 211=pion
            (df_Hc["genUp4S_PDG_1_0_2"].isna() == True)  & # no particle 
            (df_Hc["genUp4S_PDG_1_0_0_0"].abs() == 321.0)  & # 321=kaon 
            (df_Hc["genUp4S_PDG_1_0_0_1"].abs() == 211.0)  & # 211=pion
            ((df_Hc["genUp4S_PDG_1_0_0_2"].abs() == 211.0)  | # 211=pion
            (df_Hc["genUp4S_PDG_1_0_0_2"].isna() == True) | # OR no particle
            (df_Hc["genUp4S_PDG_1_0_0_2"].abs() == 111.0)) & # OR 111=pi0
            ((df_Hc["genUp4S_PDG_1_0_0_3"].abs() == 211.0)  | # 211=pion
            (df_Hc["genUp4S_PDG_1_0_0_3"].isna() == True)) & # OR no particle
            ((df_Hc["genUp4S_PDG_1_0_0_4"].abs() == 111.0)  | # 111=pi0
            (df_Hc["genUp4S_PDG_1_0_0_4"].isna() == True)) & # OR no particle
            (df_Hc["genUp4S_PDG_1_0_0_5"].isna() == True)) # no particle
     ]

In [23]:
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Hc["__event__"])]

In [24]:
print(df_FSPs.shape[0])
print(df_Hc.shape[0])

13170828
823879


## delete evts where Hc mother and Bsig overlap

In [33]:
print(df_Hc.shape[0])

823879


In [34]:
# these events have Hc as a Bsig daughter, which is of course wrong
df_Hc[df_Hc['Bsig_uniqParID'] == df_Hc['mcMother0_uniqParID']].shape[0]

623758

In [35]:
df_Hc = df_Hc[df_Hc['Bsig_uniqParID'] != df_Hc['mcMother0_uniqParID']]

In [36]:
print(df_Hc.shape[0])

200121


In [37]:
print(df_FSPs.shape[0])
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Hc["__event__"])]
print(df_FSPs.shape[0])

13170828
3397787


## add cols

In [26]:
# function to create col with the particles mother B's uniqueParticleIdentifier
df_FSPs['B_ID'] = df_FSPs.apply(B_ID, axis=1)

In [27]:
df_FSPs['B_ID'].value_counts()

83886082    5714868
83886081    5685322
0           1770638
Name: B_ID, dtype: int64

In [28]:
df_FSPs['PDG'].value_counts()

  22.00    7042518
 211.00    1545562
-211.00    1454814
  13.00     645165
 -13.00     634193
 321.00     531156
-321.00     530244
 -11.00     396066
  11.00     391110
Name: PDG, dtype: int64

In [29]:
df_Hc['PDG'].value_counts()

  413.00    348930
 -413.00    339210
 -411.00     45849
  411.00     45790
  421.00     17481
 -421.00     11497
 -431.00      4056
  431.00      3969
  423.00      2346
  443.00      1875
 -423.00      1528
 4122.00       698
-4122.00       650
Name: PDG, dtype: int64

## check D*lnu

In [31]:
groupsHc = pd.DataFrame({'count' : df_Hc[df_Hc["Bsig_uniqParID"] == 83886082.0].groupby( 
    ["genUp4S_PDG_1_0","genUp4S_PDG_1_1","genUp4S_PDG_1_2"] ).size()}).reset_index()

groupsHc

,genUp4S_PDG_1_0,genUp4S_PDG_1_1,genUp4S_PDG_1_2,count
0,-413.00,-13.00,14.00,96025
1,-413.00,-11.00,12.00,96792
2,413.00,11.00,-12.00,98459
3,413.00,13.00,-14.00,99435


In [32]:
groupsHc = pd.DataFrame({'count' : df_Hc[df_Hc["Bsig_uniqParID"] == 83886081.0].groupby( 
    ["genUp4S_PDG_0_0","genUp4S_PDG_0_1","genUp4S_PDG_0_2"] ).size()}).reset_index()

groupsHc

,genUp4S_PDG_0_0,genUp4S_PDG_0_1,genUp4S_PDG_0_2,count
0,-413.00,-13.00,14.00,106931
1,-413.00,-11.00,12.00,107626
2,413.00,11.00,-12.00,109186
3,413.00,13.00,-14.00,109425


## take care of duplis in df_Hc, delete them, because with the prior, duplis here mean there must have been an error

In [38]:
groupsHc_uniqParID = pd.DataFrame({'count' : df_Hc.groupby( ["__event__"] ).size()}).reset_index()
groupsHc_uniqParID.value_counts('count')

count
1    192318
2      3774
3        74
4         7
5         1
dtype: int64

In [39]:
df_Hc[df_Hc['__event__'].isin(groupsHc_uniqParID[groupsHc_uniqParID["count"]>1]["__event__"])][["__event__",
    "uniqueParticleIdentifier","PDG",
    "genMothPDG_0",'Bsig_uniqParID',"mcMother0_uniqParID","px","E","x","genUp4S_PDG_1_0"]][:10]

,__event__,uniqueParticleIdentifier,PDG,genMothPDG_0,Bsig_uniqParID,mcMother0_uniqParID,px,E,x,genUp4S_PDG_1_0
232,7754384,100663296.00,-421.00,511.00,83886082.00,83886081.00,0.47,2.23,0.60,413.00
683,30923832,100663296.00,421.00,-511.00,83886082.00,83886081.00,0.29,2.23,-0.04,413.00
701,42325767,100663296.00,421.00,-511.00,83886082.00,83886081.00,1.15,2.32,0.00,413.00
702,42325767,100663296.00,421.00,-511.00,83886082.00,83886081.00,-0.61,2.31,0.00,413.00
703,42325872,100663296.00,-421.00,511.00,83886081.00,83886082.00,0.55,2.32,0.15,421.00
704,42325872,100663296.00,421.00,511.00,83886081.00,83886082.00,0.28,2.25,-0.05,421.00
730,39480850,100663296.00,-421.00,-511.00,83886081.00,83886082.00,0.14,1.96,-0.06,-421.00
748,38974952,100663296.00,421.00,511.00,83886081.00,83886082.00,-0.29,2.04,-0.05,-411.00
779,40137428,100663296.00,-421.00,511.00,83886082.00,83886081.00,0.07,1.97,-0.03,-413.00
832,38948866,100663296.00,421.00,511.00,83886082.00,83886081.00,0.43,2.00,-0.05,413.00


In [40]:
df_Hc = df_Hc[df_Hc['__event__'].isin(groupsHc_uniqParID[groupsHc_uniqParID["count"]==1]["__event__"])]

In [41]:
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Hc["__event__"])]

In [42]:
groupsHc_uniqParID = pd.DataFrame({'count' : df_Hc.groupby( ["__event__"] ).size()}).reset_index()
groupsHc_uniqParID.value_counts('count')

count
1    192318
dtype: int64

## delete the H_c FSPs 

In [43]:
df_FSPs['ancestor_below_B_arrayID'] = df_FSPs.apply(ancestor_below_B, axis=1)

In [44]:
df_Hc["HcMother_uniqParID"] = df_Hc["mcMother0_uniqParID"]
df_Hc["Hc_genParticleID"] = df_Hc["genParticleID"]
df_Hc["Hc_MotherPDG"] = df_Hc["genMothPDG_0"]

In [45]:
df_FSPs = pd.merge(df_FSPs,df_Hc[["__event__","Bsig_uniqParID","HcMother_uniqParID","Hc_genParticleID","Hc_MotherPDG"]], on="__event__")

In [46]:
df_FSPs[["__event__","Bsig_uniqParID","HcMother_uniqParID","Hc_genParticleID",
         "ancestor_below_B_arrayID","Hc_MotherPDG"]][:10]

,__event__,Bsig_uniqParID,HcMother_uniqParID,Hc_genParticleID,ancestor_below_B_arrayID,Hc_MotherPDG
0,2332321,83886082.00,83886081.00,3.00,6.00,511.00
1,2332321,83886082.00,83886081.00,3.00,6.00,511.00
2,2332321,83886082.00,83886081.00,3.00,5.00,511.00
3,2332321,83886082.00,83886081.00,3.00,0.00,511.00
4,2332321,83886082.00,83886081.00,3.00,0.00,511.00
5,2332321,83886082.00,83886081.00,3.00,3.00,511.00
6,2332321,83886082.00,83886081.00,3.00,4.00,511.00
7,2332321,83886082.00,83886081.00,3.00,0.00,511.00
8,2332321,83886082.00,83886081.00,3.00,3.00,511.00
9,2332321,83886082.00,83886081.00,3.00,3.00,511.00


In [47]:
df_FSPs.shape[0]

3326989

In [48]:
# deletion of Hc FSPs, those who 
df_FSPs = df_FSPs[~((df_FSPs["Hc_genParticleID"] == df_FSPs["ancestor_below_B_arrayID"]) &  # originate from Hc arrayID(genParticleID)
                  (df_FSPs["HcMother_uniqParID"] == df_FSPs["B_ID"]))] # and are B-tag ancestors

In [49]:
df_FSPs.shape[0]

2497500

## delete delete_top_numFSP_evts

In [50]:
numberFSPsEvts = pd.DataFrame({'count' : df_FSPs.groupby(["__event__"]).size()}).reset_index()
numEvtsBeforeCut = df_Hc.shape[0]

In [51]:
# print first to adjust percentile used
print("original max FSPs:",numberFSPsEvts["count"].max())
for percentile_lvl in [0.995,0.996,0.997,0.998,0.999]:
    print(percentile_lvl)
    percentile = numberFSPsEvts["count"].quantile(q=percentile_lvl)
    cut_numberFSPsEvts = numberFSPsEvts[numberFSPsEvts["count"] < percentile]
    
    #TMP_df_FSPs = df_FSPs[df_FSPs['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    TMP_df_Hc = df_Hc[df_Hc['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    
    print("would delete", round(100 - (TMP_df_Hc.shape[0]/df_Hc.shape[0])*100,2),"% of evts")
    print("new max FSPs:",cut_numberFSPsEvts["count"].max())

original max FSPs: 49
0.995
would delete 0.61 % of evts
new max FSPs: 32
0.996
would delete 0.48 % of evts
new max FSPs: 33
0.997
would delete 0.38 % of evts
new max FSPs: 34
0.998
would delete 0.21 % of evts
new max FSPs: 36
0.999
would delete 0.11 % of evts
new max FSPs: 38


In [49]:
print(df_FSPs.shape[0])
print(df_Hc.shape[0])


if delete_top_numFSP_evts:
    delete_percentileLvl = 0.995
    print("used delete_percentileLvl:",delete_percentileLvl)
    
    
    print(numberFSPsEvts["count"].describe())
    
    percentile = numberFSPsEvts["count"].quantile(q=delete_percentileLvl)
    cut_numberFSPsEvts = numberFSPsEvts[numberFSPsEvts["count"] < percentile]
    
    df_FSPs = df_FSPs[df_FSPs['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    df_Hc = df_Hc[df_Hc['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    
print(df_FSPs.shape[0])
print(df_Hc.shape[0])
print("deleted", round(100 - (df_Hc.shape[0]/numEvtsBeforeCut)*100,4),"% of events")
numberFSPsEvts = pd.DataFrame({'count' : df_FSPs.groupby(["__event__"]).size()}).reset_index()
print(numberFSPsEvts["count"].describe())

11163573
828867
used delete_percentileLvl: 0.995
count    828867.000000
mean         13.468473
std           3.960402
min           1.000000
25%          11.000000
50%          13.000000
75%          16.000000
max          35.000000
Name: count, dtype: float64
10996905
822214
deleted 9901.0 % of events
count    822214.000000
mean         13.374748
std           3.834275
min           1.000000
25%          11.000000
50%          13.000000
75%          16.000000
max          23.000000
Name: count, dtype: float64


## check that each particles occurs only ones; count must be <= 1

In [52]:
groupsFSPs_uniqParID = pd.DataFrame({'count' : df_FSPs.groupby( ["__event__","uniqueParticleIdentifier"] ).size()}).reset_index()
groupsFSPs_uniqParID.value_counts('count')

count
1    2448774
2      24363
dtype: int64

## impute the nan values of variables

In [50]:
## data preprocessing, dealing with NaN
imputelist = [["pionID",-1.],["kaonID",-1.],["electronID",-1.],["muonID",-1.],["protonID",-1.]
              ,["clusterReg",-1.],["clusterE9E21",-1.]
              ]
for impute in imputelist:
    column_name = impute[0]
    impute_val = impute[1]
    mask = df_FSPs[column_name].isna() == 1
    df_FSPs.loc[mask, column_name] = impute_val
    print(df_FSPs[df_FSPs[column_name].isna() == True].shape[0])

0
0
0
0
0
0
0


## last check if events match between dfs

In [51]:
df_Hc = df_Hc[df_Hc['__event__'].isin(df_FSPs["__event__"])]
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Hc["__event__"])]

In [52]:
print(df_FSPs.shape[0])
print(df_Hc.shape[0])

10996905
822214


## merge x,y,z  vertex info of Hc to FSPs

In [140]:
for coordinate in ["x","y","z"]:
    print(df_Hc[coordinate].describe())    
    print(df_Hc[f'{coordinate}_uncertainty'].describe())
    print("\n")
    

count               96244.00
mean                   -0.04
std                     0.39
min                   -16.84
25%                    -0.06
50%                    -0.05
75%                    -0.02
max                    20.43
Name: x, dtype: float64
count               96244.00
mean                    0.05
std                     0.20
min                     0.00
25%                     0.00
50%                     0.00
75%                     0.02
max                    24.46
Name: x_uncertainty, dtype: float64


count               96244.00
mean                    0.02
std                     0.39
min                   -18.44
25%                     0.00
50%                     0.02
75%                     0.03
max                    17.68
Name: y, dtype: float64
count               96244.00
mean                    0.05
std                     0.24
min                     0.00
25%                     0.00
50%                     0.00
75%                     0.02
max            

In [145]:
df_FSPs = pd.merge(df_FSPs,df_Hc[["__event__","x","x_uncertainty","y","y_uncertainty","z","z_uncertainty"]], on="__event__")

## save the dfs as they will be used for data prod

In [146]:
df_FSPs_final = df_FSPs
df_Y4S_final = df_Hc

In [147]:
root_path

'/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/NAHS_allEvts_oneSub/'

In [148]:
df_FSPs_final.to_csv(root_path + sub + "sub_df_FSPs_preProcessed_NAHS.csv")
df_Y4S_final.to_csv(root_path + sub + "sub_df_Y4S_preProcessed_NAHS.csv")

## load the df's and concat them

In [6]:
dfs0=[]
dfs1=[]

for sub in ["sub0/","sub1/"]:
    tmp_df_FSPs = pd.read_csv(root_path + sub + "sub_df_FSPs_preProcessed_NAHS.csv".format(subset_size))
    tmp_df_Y4S = pd.read_csv(root_path + sub + "sub_df_Y4S_preProcessed_NAHS.csv")
    
    dfs0.append(tmp_df_FSPs)
    dfs1.append(tmp_df_Y4S)

In [ ]:
df_FSPs_final = pd.concat(dfs0)
df_Y4S_final = pd.concat(dfs1)

In [148]:
df_FSPs_final.to_csv(root_path + "final_df_FSPs_preProcessed_NAHS.csv")
df_Y4S_final.to_csv(root_path + "final_df_Y4S_preProcessed_NAHS.csv")

## load final df's 

In [6]:
df_FSPs_final = pd.read_csv(root_path + "final_df_FSPs_preProcessed_NAHS.csv".format(subset_size))
df_Y4S_final = pd.read_csv(root_path + "final_df_Y4S_preProcessed_NAHS.csv")

## beginning of data prod

In [126]:
numFSPs = pd.DataFrame({'count' : df_FSPs_final.groupby( ["__event__"] ).size()}).reset_index()

minFSPs = numFSPs["count"].min()
maxFSPs = numFSPs["count"].max()
print("minFSPs:",minFSPs)
print("maxFSPs:",maxFSPs,'\n')
print("num Events:",numFSPs.shape[0],'\n')

df_FSPs_final['numFSPs'] = df_FSPs_final.groupby('__event__')['__event__'].transform('count')

minFSPs: 1
maxFSPs: 23 

num Events: 96244 



In [149]:
variable_combinations = [
    [[    "charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID",
     "x","y","z"],"NAHSA_no_M"],
    [["M",         "dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID",
     "x","y","z"],"NAHSA_no_charge"],
    [["M","charge",     "dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID",
     "x","y","z"],"NAHSA_no_dr"],
    [["M","charge","dr",     "clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID",
     "x","y","z"],"NAHSA_no_dz"],
    [["M","charge","dr","dz",             "clusterE9E21","pionID","kaonID","electronID","muonID","protonID",
     "x","y","z"],"NAHSA_no_clReg"],
    [["M","charge","dr","dz","clusterReg",               "pionID","kaonID","electronID","muonID","protonID",
     "x","y","z"],"NAHSA_no_E9E21"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21",         "kaonID","electronID","muonID","protonID",
     "x","y","z"],"NAHSA_no_pID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID",         "electronID","muonID","protonID",
     "x","y","z"],"NAHSA_no_kID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID",             "muonID","protonID",
     "x","y","z"],"NAHSA_no_eID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID"         ,"protonID",
     "x","y","z"],"NAHSA_no_muID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID"           ,
     "x","y","z"],"NAHSA_no_proID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID"
                 ],"NAHSA_no_xyz"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID",
     "x","y","z"],"NAHSA_allExtras"],    
    [["charge","pionID","muonID","x","y","z"],"NAHSA_charge_mpIDs_xyz"],
    #[[],"NAHSA_only4mom"],
    [["charge"],"NAHSA_charge"]
]

In [127]:
variable_combinations = [
    [[],"NAHSA_only4mom"]
]

In [150]:
minFSPs = 5 # minimum: Bsig: K pi pi lepton, X: 1 

In [151]:
for var_comb in variable_combinations:
    ablationSub = var_comb[1]
    data_dir = Path(nfs_path + "data/" + data_subdir + root_subdir + ablationSub + "/")    
    if save_data:
        data_dir.mkdir(parents=True, exist_ok=True)
    print("Will save data to:", data_dir,'is', save_data ,'\n')


    
    for num_FSPs_toData in range(minFSPs, maxFSPs+1):
        df_num_subset = df_FSPs_final.copy()
        df_num_subset = df_num_subset[df_num_subset['numFSPs'] == num_FSPs_toData]


        numEvents = df_num_subset.__event__.nunique()
        print("numEvents:",numEvents)
        print("num_FSPs_toData:",num_FSPs_toData)  
        if numEvents == 0:
            print("skipped because empty \n")
            continue

        if numEvents < 10:
            print("skipped because <10 events \n")
            continue

        num_features = 4 + len(var_comb[0])
        leaves = np.zeros((numEvents, num_FSPs_toData,  num_features))  
        SA_target =  np.zeros((numEvents, num_FSPs_toData))
        global_tag = np.chararray((numEvents, num_FSPs_toData + 1), itemsize=30)

        event_list = df_num_subset[df_num_subset["numFSPs"] == num_FSPs_toData]["__event__"].unique()
        #print("len(event_list):",len(event_list))
        for i in range(numEvents):

            event_iter = event_list[i]

            global_tag_masterInfo = "evt" + str(event_iter)
            global_tag[i,-1] = global_tag_masterInfo
            #print("global_tag[i,-1]:",global_tag[i,-1])
            #print("i:",i,"event_iter:",event_iter)

            event_df = df_num_subset[df_num_subset.__event__ == event_iter]

            for j in range(num_FSPs_toData):
                #print("numParticle:",j)
                particle = event_df.iloc[j]

                #print(particle["mcPDG"],particle["px"],particle["py"],particle["pz"],particle["E"])
                leaves[i,j,0] = particle["px"]
                leaves[i,j,1] = particle["py"]
                leaves[i,j,2] = particle["pz"]
                leaves[i,j,3] = particle["E"]
                
                for k in range(len(var_comb[0])):
                    leav_idx = k + 4
                    leaves[i,j,leav_idx] = particle[var_comb[0][k]]
                    



                global_tag_Info = str((particle["mcPDG"])) 
                global_tag[i,j] = global_tag_Info

                label = -10 # error code if assignment fails
                B_tag_uniqID = -10 # error code if assignment fails     

                B_sig_uniqID = df_Y4S_final[df_Y4S_final["__event__"] == event_iter].iloc[0]['Bsig_uniqParID']
                if B_sig_uniqID == 83886082.0:
                    B_tag_uniqID = 83886081.0
                elif B_sig_uniqID == 83886081.0:
                    B_tag_uniqID = 83886082.0

                if particle["B_ID"] == B_tag_uniqID:
                    label = 1 # particle belongs to X (MC truth)
                elif particle["B_ID"] == B_sig_uniqID:
                    label = 2 # particle belongs to Bsig (MC truth)
                elif particle["B_ID"] == 0:
                    label = 0 # background (MC truth)


                SA_target[i,j] = label

            del event_df


        # shuffle the data    
        for idx in np.arange(leaves.shape[0]):   # arange is like range but gives ndarray instead of list
            perms = np.random.permutation(leaves.shape[1])

            leaves[idx,:] = leaves[idx,perms]
            SA_target[idx,:] = SA_target[idx,perms]
            global_tag[idx,0:-1] = global_tag[idx,perms]





        #print(global_tag)
        train_ratio = 0.8
        validation_ratio = 0.2
        #test_ratio = 0.1

        print("leaves.shape:",leaves.shape)
        print("SA_target.shape:",SA_target.shape)
        print("global_tag.shape:",global_tag.shape)


        print("leaves[0]:",leaves[0])
        print("SA_target[0]:",SA_target[0])
        print("global_tag[0]:",global_tag[0])

        x=leaves
        y=SA_target
        z=global_tag

        x_train, x_val, y_train, y_val, z_train, z_val = train_test_split(x, y, z, test_size=1 - train_ratio, shuffle=False)
        #x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(x, y, z, test_size=1 - train_ratio, shuffle=False)
        #x_val, x_test, y_val, y_test, z_val, z_test = train_test_split(x_test, y_test, z_test, test_size=test_ratio/(test_ratio + validation_ratio), shuffle=False) 

        if save_data==True:
            np.save(data_dir / "leaves_train_FSP{}.npy".format(num_FSPs_toData), x_train)
            np.save(data_dir / "is_left_arr_train_FSP{}.npy".format(num_FSPs_toData), y_train)
            np.save(data_dir / "global_tag_train_FSP{}.npy".format(num_FSPs_toData), z_train)

            np.save(data_dir / "leaves_val_FSP{}.npy".format(num_FSPs_toData), x_val)
            np.save(data_dir / "is_left_arr_val_FSP{}.npy".format(num_FSPs_toData), y_val)
            np.save(data_dir / "global_tag_val_FSP{}.npy".format(num_FSPs_toData), z_val)

            #np.save(data_dir / "leaves_test_FSP{}.npy".format(num_FSPs_toData), x_test)
            #np.save(data_dir / "is_left_arr_test_FSP{}.npy".format(num_FSPs_toData), y_test)
            #np.save(data_dir / "global_tag_test_FSP{}.npy".format(num_FSPs_toData), z_test)


        print("")


        del df_num_subset


Will save data to: /nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/data/NAHSA_Gmodes_1/NAHS_allEvts_oneSub/NAHSA_no_M is True 

numEvents: 1232
num_FSPs_toData: 5
leaves.shape: (1232, 5, 14)
SA_target.shape: (1232, 5)
global_tag.shape: (1232, 6)
leaves[0]: [[ 0.14259353 -0.23874733 -0.42714393  0.50969122  0.          0.
   0.          3.          0.91699219 -1.         -1.         -1.
  -1.         -1.        ]
 [-0.79012734  0.27009329  0.09869234  0.97504294 -1.          0.00682038
   0.00862561  2.          0.98535156  0.00096549  0.15175247  0.84725974
   0.00000072  0.00002158]
 [-0.30713138 -0.09685621 -0.17984667  0.39437993  1.          0.00119517
   0.00537646  2.          0.92382812  0.91642372  0.          0.05465339
   0.02892289  0.        ]
 [-0.32595828  0.89728135  0.53782165  1.09572579 -1.          0.00434325
   0.01232462  2.          0.98046875  0.0000001   0.          0.9999999
   0.          0.        ]
 [ 0.08999581 -0.10610624 -0.10206289  0.1725532


numEvents: 9179
num_FSPs_toData: 11
leaves.shape: (9179, 11, 14)
SA_target.shape: (9179, 11)
global_tag.shape: (9179, 12)
leaves[0]: [[  0.11262705   0.38826191  -0.04237363   0.4297762    1.
    0.00503318   0.01779051   2.           0.828125     0.99999338
    0.           0.           0.00000662   0.        ]
 [  0.11877039  -0.04846106   0.0537724    0.13909116   0.
    0.           0.           2.           0.96484375  -1.
   -1.          -1.          -1.          -1.        ]
 [  0.45995867   0.34882525   0.55719155   0.8143607    1.
    0.02532729   0.0096378    2.           0.94824219   0.65201295
    0.           0.           0.34798705   0.        ]
 [  0.05562339  -0.21132517   0.04163673   0.2626134   -1.
    0.91074547 -19.89052408  -1.          -1.           0.95649555
    0.00000001   0.00000045   0.04350399   0.        ]
 [ -0.0667891    0.00932067  -0.04959819   0.08371164   0.
    0.           0.           2.           0.97460938  -1.
   -1.          -1.          -1.


numEvents: 7940
num_FSPs_toData: 15
leaves.shape: (7940, 15, 14)
SA_target.shape: (7940, 15)
global_tag.shape: (7940, 16)
leaves[0]: [[ 0.31945494  0.64409441 -0.66827637  0.98725177 -1.          0.00328105
  -0.00342831  3.          0.93261719  0.00000002  0.          0.
   0.99999998  0.        ]
 [-0.10501229  0.33442786  0.27023965  0.46408936  1.          0.0046555
  -0.01145058  2.          0.71777344  0.97778751  0.          0.
   0.02221249  0.        ]
 [-0.40171927 -0.06258156 -0.03385902  0.64043597 -1.          0.01168251
   0.00188068  2.          0.9296875   0.          0.99954766  0.00045234
   0.          0.        ]
 [-0.09336021 -0.11915761  0.0151157   0.20645349  1.          0.00395096
  -0.01071097 -1.         -1.          0.79141541  0.          0.12878226
   0.07980232  0.        ]
 [-0.06779338 -0.07801262  0.25145912  0.27187056  0.          0.
   0.          1.          0.93359375 -1.         -1.         -1.
  -1.         -1.        ]
 [-0.32879299  0.4547351


numEvents: 3619
num_FSPs_toData: 18
leaves.shape: (3619, 18, 14)
SA_target.shape: (3619, 18)
global_tag.shape: (3619, 19)
leaves[0]: [[ 1.73999488  0.36547542 -0.1942088   1.79165712 -1.          0.00015737
   0.03236121  2.          1.          0.          0.          0.
   1.          0.        ]
 [-0.21438807 -0.09312129  0.06123922  0.26371908  1.          0.00117293
   0.03220516 -1.         -1.          0.41952432  0.          0.10854576
   0.47192992  0.        ]
 [ 0.09446815 -0.28904667  0.11079688  0.3404583  -1.          4.9863678
  -0.05844821  2.          0.85058594  0.00015386  0.          0.
   0.99984614  0.        ]
 [ 0.14279842 -0.03570288  0.02649292  0.20456748  1.          0.01077671
   0.02555076 -1.         -1.          0.50937439  0.          0.26131615
   0.22930946  0.        ]
 [-0.14482288  0.28217128  0.32645831  0.67148067  1.          0.00233556
   0.03633202  2.          0.96289062  0.          1.          0.
   0.          0.        ]
 [ 0.00356749  0


numEvents: 899
num_FSPs_toData: 21
leaves.shape: (899, 21, 14)
SA_target.shape: (899, 21)
global_tag.shape: (899, 22)
leaves[0]: [[ 0.14286678 -0.06726087  0.27888694  0.32048848  0.          0.
   0.          1.          0.96484375 -1.         -1.         -1.
  -1.         -1.        ]
 [-0.03520446  0.08580355  0.23485889  0.25250802  0.          0.
   0.          1.          0.98046875 -1.         -1.         -1.
  -1.         -1.        ]
 [-0.6345256   0.30085239 -0.43105927  0.83571938  1.          0.00621468
   0.06154096  2.          1.          0.48297474  0.21374292  0.00019239
   0.30308996  0.        ]
 [ 0.00567335  0.09871063  0.08316176  0.12919696  0.          0.
   0.          2.          0.98632812 -1.         -1.         -1.
  -1.         -1.        ]
 [-0.02272872 -0.08711375  0.09272359  0.12924033  0.          0.
   0.          2.          0.91210938 -1.         -1.         -1.
  -1.         -1.        ]
 [ 0.02518787 -0.19762237 -0.18241127  0.29004585 -1.      

...


numEvents: 899
num_FSPs_toData: 21
leaves.shape: (899, 21, 10)
SA_target.shape: (899, 21)
global_tag.shape: (899, 22)
leaves[0]: [[ 0.10690476  0.08279679 -0.15073922  0.2025     -0.08380046  0.00139985
   0.00411554  0.00189685  0.13418488  0.00134936]
 [ 0.02518787 -0.19762237 -0.18241127  0.29004585 -0.08380046  0.00139985
   0.00411554  0.00189685  0.13418488  0.00134936]
 [-0.04792266 -0.02597603  0.03677687  0.06575617 -0.08380046  0.00139985
   0.00411554  0.00189685  0.13418488  0.00134936]
 [ 0.07425193 -0.03471493 -0.03859795  0.09059954 -0.08380046  0.00139985
   0.00411554  0.00189685  0.13418488  0.00134936]
 [-0.6345256   0.30085239 -0.43105927  0.83571938 -0.08380046  0.00139985
   0.00411554  0.00189685  0.13418488  0.00134936]
 [ 0.1882955   0.18576707  0.03839356  0.26728012 -0.08380046  0.00139985
   0.00411554  0.00189685  0.13418488  0.00134936]
 [ 0.24308966 -0.1635264   0.17794104  0.34277798 -0.08380046  0.00139985
   0.00411554  0.00189685  0.13418488  0.00134


Will save data to: /nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/data/NAHSA_Gmodes_1/NAHS_allEvts_oneSub/NAHSA_Hc_xyz is True 

numEvents: 1232
num_FSPs_toData: 5
leaves.shape: (1232, 5, 7)
SA_target.shape: (1232, 5)
global_tag.shape: (1232, 6)
leaves[0]: [[ 0.14259353 -0.23874733 -0.42714393  0.50969122  0.          0.
   0.        ]
 [-0.30713138 -0.09685621 -0.17984667  0.39437993  0.          0.
   0.        ]
 [-0.32595828  0.89728135  0.53782165  1.09572579  0.          0.
   0.        ]
 [-0.79012734  0.27009329  0.09869234  0.97504294  0.          0.
   0.        ]
 [ 0.08999581 -0.10610624 -0.10206289  0.17255322  0.          0.
   0.        ]]
SA_target[0]: [2. 2. 2. 2. 2.]
global_tag[0]: [b'22.0' b'211.0' b'11.0' b'-321.0' b'22.0' b'evt3679068']

numEvents: 2597
num_FSPs_toData: 6
leaves.shape: (2597, 6, 7)
SA_target.shape: (2597, 6)
global_tag.shape: (2597, 7)
leaves[0]: [[-0.80670559 -0.10175595  0.95644319  1.25535324 -0.02159487  0.00379263
  -0.0060693 ]



numEvents: 9734
num_FSPs_toData: 13
leaves.shape: (9734, 13, 7)
SA_target.shape: (9734, 13)
global_tag.shape: (9734, 14)
leaves[0]: [[-0.06911616 -0.01208754  0.01307649  0.07137329 -0.07433347 -0.03684826
   0.0138246 ]
 [-0.08442178  0.08756821 -0.04758492  0.13061222 -0.07433347 -0.03684826
   0.0138246 ]
 [ 0.46631134  0.72483313  0.25450766  0.89866761 -0.07433347 -0.03684826
   0.0138246 ]
 [-0.38076156 -0.10995463  0.3469387   0.52672179 -0.07433347 -0.03684826
   0.0138246 ]
 [ 0.2776275  -0.22842607  0.22580674  0.42455205 -0.07433347 -0.03684826
   0.0138246 ]
 [ 0.08953707  0.00481639  0.05675516  0.10611896 -0.07433347 -0.03684826
   0.0138246 ]
 [-0.30117601 -0.08275584  0.28913125  0.65182005 -0.07433347 -0.03684826
   0.0138246 ]
 [-0.01618002 -0.10223757  0.06339474  0.12138042 -0.07433347 -0.03684826
   0.0138246 ]
 [-0.06117035 -0.05394757 -0.06264355  0.10284146 -0.07433347 -0.03684826
   0.0138246 ]
 [-0.95928693 -0.45345503  0.7602697   1.30532099 -0.07433347 -0.0


numEvents: 3619
num_FSPs_toData: 18
leaves.shape: (3619, 18, 7)
SA_target.shape: (3619, 18)
global_tag.shape: (3619, 19)
leaves[0]: [[ 0.09446815 -0.28904667  0.11079688  0.3404583  -0.00592267  0.14631526
  -0.06511321]
 [-0.10357299 -0.00602906  0.04720151  0.11398112 -0.00592267  0.14631526
  -0.06511321]
 [ 0.00356749  0.30565831  0.14877877  0.33996301 -0.00592267  0.14631526
  -0.06511321]
 [-0.09047557  0.15642798  0.04320596  0.18580177 -0.00592267  0.14631526
  -0.06511321]
 [ 0.03601022  0.07479906 -0.0159031   0.08452541 -0.00592267  0.14631526
  -0.06511321]
 [-0.02135795 -0.07924473  0.08120764  0.18113659 -0.00592267  0.14631526
  -0.06511321]
 [-0.14482288  0.28217128  0.32645831  0.67148067 -0.00592267  0.14631526
  -0.06511321]
 [ 0.02964692 -0.21867526  0.15798448  0.27139805 -0.00592267  0.14631526
  -0.06511321]
 [ 0.14279842 -0.03570288  0.02649292  0.20456748 -0.00592267  0.14631526
  -0.06511321]
 [ 0.10279583 -0.00816721  0.04297194  0.11171514 -0.00592267  0.1


numEvents: 525
num_FSPs_toData: 22
leaves.shape: (525, 22, 7)
SA_target.shape: (525, 22)
global_tag.shape: (525, 23)
leaves[0]: [[ 0.01843309  0.09318832  0.0339824   0.10088927 -0.04778086  0.01618314
   0.00084682]
 [-0.27001593 -0.16322896 -0.05838112  0.34991505 -0.04778086  0.01618314
   0.00084682]
 [ 0.11785215  0.05715174 -0.01313976  0.16879513 -0.04778086  0.01618314
   0.00084682]
 [-0.05709071  0.28131112  0.00515626  0.3059176  -0.04778086  0.01618314
   0.00084682]
 [ 0.21023762  0.12959823 -0.06993845  0.27757996 -0.04778086  0.01618314
   0.00084682]
 [-0.11816879  0.07288201  0.33107433  0.35900677 -0.04778086  0.01618314
   0.00084682]
 [-0.23025732  0.50089377  0.16686134  0.57598261 -0.04778086  0.01618314
   0.00084682]
 [ 1.48257625 -0.19425921  0.27385363  1.59827462 -0.04778086  0.01618314
   0.00084682]
 [ 0.02968633 -0.03962716 -0.0312976   0.05857585 -0.04778086  0.01618314
   0.00084682]
 [ 0.18668237  0.00067523 -0.05878216  0.19571946 -0.04778086  0.01618

In [152]:
print("saving is done")
now = datetime.now()
print("time at end =", now)

saving is done
time at end = 2021-10-26 19:18:34.991419
